# <CENTER> DATA DE MATRÍCULA

## *PASOS INICIALES*

### *LIBRERÍAS*

In [37]:
import pandas as pd
import numpy as np
import unidecode
import os
import openpyxl
import datetime
from datetime import date, datetime, timedelta
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import re

pd.set_option('display.max_columns', None)

### *RUTAS*

In [43]:
ruta_matricula_bronze = '../data/matricula_bronze/'
ruta_matricula_gold = '../data/matricula_gold/'

ruta_output_pq = '../output/ds_matricula.parquet'

### *FUNCIONES*

In [31]:
def LimpiarNumero( numero_str ) :
    try : 
        numero = float( numero_str )
    except :
        try :
            numero = float( str( numero_str ).replace( ',' , '.' ) )
        except :
            numero = np.nan
    return numero

### *DATOS A ESPECIFICAR*

In [35]:
lista_periodos = [
    202401 ,
    202402 ,
    202501 ,
    202502 ,
]

# CARGAR DATA

## BRONZE

In [32]:
cols = [
    'COD_ALUMNO' , 
    'CREDITOS_APROBADOS_ACUMULADOS' , 
    'CREDITOS_TOTALES_CARRERA' , 
    'CANTIDAD_CURSOS_MATRICULADOS' , 
    'DOCUMENTO_ALUMNO' , 
    'NRO_PONDERADO_ACUMULADO' , 
    'PONDERADO_CICLO_ANTERIOR' ,
]
conv = {
    'COD_ALUMNO' : str , 
    'DOCUMENTO_ALUMNO' : str , 
    'CREDITOS_APROBADOS_ACUMULADOS' : str , 
    'CREDITOS_TOTALES_CARRERA' : str , 
    'NRO_PONDERADO_ACUMULADO' : str , 
    'PONDERADO_CICLO_ANTERIOR' : str ,
}
rename = {
    'COD_ALUMNO' : 'cod_alumno' ,
    'CREDITOS_APROBADOS_ACUMULADOS' : 'cant_creds_aprob_acum' , 
    'CREDITOS_TOTALES_CARRERA' : 'cant_creds_carrera' , 
    'CANTIDAD_CURSOS_MATRICULADOS' : 'cant_cursos_matr' , 
    'DOCUMENTO_ALUMNO' : 'documento' , 
    'NRO_PONDERADO_ACUMULADO' : 'ponderado_acum' , 
    'PONDERADO_CICLO_ANTERIOR' : 'ponderado_ant' ,
}

lista_dfs = []
for arch in os.listdir( ruta_matricula_bronze ) :
    df_matr_b = pd.read_csv( ruta_matricula_bronze + arch , sep = '|' , usecols = cols , converters = conv )
    df_matr_b.rename( rename , axis = 1 , inplace = True )
    df_matr_b['periodo'] = int( arch[ -10 : -4 ] )
    lista_dfs.append( df_matr_b )
df_matr_b = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

df_matr_b['cant_creds_aprob_acum'] = df_matr_b['cant_creds_aprob_acum'].apply( LimpiarNumero )
df_matr_b['cant_creds_carrera'] = df_matr_b['cant_creds_carrera'].apply( LimpiarNumero )
df_matr_b['ponderado_acum'] = df_matr_b['ponderado_acum'].apply( LimpiarNumero )
df_matr_b['ponderado_ant'] = df_matr_b['ponderado_ant'].apply( LimpiarNumero )

df_matr_b

,cod_alumno,cant_creds_aprob_acum,cant_creds_carrera,cant_cursos_matr,documento,ponderado_acum,ponderado_ant,periodo
0,N00240983,165.0,200.0,2,77529979,18.83,18.71,202401
1,N00256833,211.0,242.0,5,72707410,16.07,0.00,202401
2,N00404113,18.0,200.0,5,72706434,17.61,17.61,202401
3,N00205705,172.0,202.0,3,72799837,14.65,0.00,202401
4,N00426135,NaN,200.0,6,70706800,NaN,NaN,202401
...,...,...,...,...,...,...,...,...
510961,N00521205,NaN,240.0,7,73906045,NaN,NaN,202502
510962,N00521232,NaN,240.0,7,40806322,NaN,NaN,202502
510963,N00521254,NaN,240.0,5,74066598,NaN,NaN,202502
510964,N00521317,NaN,240.0,7,77699069,NaN,NaN,202502


## GOLD

In [4]:
df_matr_g = pd.read_parquet( ruta_matricula_gold )

df_matr_g

,periodo,cod_alumno,fecha_corte,categoria_pago,cod_carrera,carrera,carrera_desc,cod_facultad,facultad,facultad_desc,campus,campus_desc,edad,departamento,provincia,distrito,flag_tercio_per_ant,flag_quinto_per_ant,flag_decimo_per_ant,unidad_negocio,modalidad,grupo_tipo_ingreso,es_verano,flag_mtr_per_ant,max_mtr_continuas_reg,pct_beca,flag_beca,desc_beca,n_per_mtr_hist,n_per_mtr_hist_reg,n_per_mtr_hist_verano,flag_mtr_verano,estado_matricula_new
0,202001,N00001048,2020-08-15,NO_DETERMINADO,CON-WA,CONTABILIDAD Y FINANZAS,CONT_FIN,NaN,NEGOCIOS,NEG,TML,NO_DETERMINADO,57,LA LIBERTAD,TRUJILLO,TRUJILLO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,0,1,0.65,1,TR.BECA.WA.COLABORAD.65%,1,1,0,0,NEW_00
1,202001,N00001095,2020-08-15,NO_DETERMINADO,DEH-WA,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,TML,NO_DETERMINADO,49,LA LIBERTAD,TRUJILLO,TRUJILLO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
2,202001,N00001409,2020-08-15,NO_DETERMINADO,DEH-WA,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,TML,NO_DETERMINADO,44,LA LIBERTAD,TRUJILLO,HUANCHACO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
3,202001,N00001588,2020-08-15,NO_DETERMINADO,MIN-WA,ING. DE MINAS,NO_DETERMINADO,NaN,INGENIERIA,ING,TSI,NO_DETERMINADO,42,LA LIBERTAD,TRUJILLO,LA ESPERANZA,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
4,202001,N00002040,2020-08-15,NO_DETERMINADO,CIV-WA,ING. CIVIL,NO_DETERMINADO,NaN,INGENIERIA,ING,CAJ,NO_DETERMINADO,42,CAJAMARCA,CAJAMARCA,CAJAMARCA,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399131,202502,N00521521,2026-01-15,NO_DETERMINADO,IND-WV,ING. INDUSTRIAL,ING_IND,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,22,LIMA,LIMA,LIMA/LIMA/LIMA,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
1399132,202502,N00521522,2026-01-15,NO_DETERMINADO,IND-WV,ING. INDUSTRIAL,ING_IND,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,32,LIMA,HUAURA,LIMA/HUAURA/HUACHO,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
1399133,202502,N00521523,2026-01-15,NO_DETERMINADO,CON-WV,CONTABILIDAD Y FINANZAS,CONT_FIN,NaN,NEGOCIOS,NEG,VIR,NO_DETERMINADO,20,LIMA,LIMA,LIMA/LIMA/INDEPENDENCIA,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00
1399134,202502,N00521524,2026-01-15,NO_DETERMINADO,SIC-WV,ING. DE SIST. COMPUTACIONALES,NO_DETERMINADO,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,28,LIMA,LIMA,LIMA/LIMA/SAN MARTIN DE PORRES,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.00,0,None,1,1,0,0,NEW_00


# TRATAMIENTO DE DATA

In [40]:
df = pd.merge( df_matr_g , df_matr_b , how = 'left' , on = [ 'periodo' , 'cod_alumno' ] )
df = df[ df['periodo'].isin( lista_periodos ) ].copy().reset_index( drop = True )

df['avance_creds_carrera'] = df['cant_creds_aprob_acum'] / df['cant_creds_carrera']
df['avance_creds_carrera'] = df['avance_creds_carrera'].fillna( 0 )

df

,periodo,cod_alumno,fecha_corte,categoria_pago,cod_carrera,carrera,carrera_desc,cod_facultad,facultad,facultad_desc,campus,campus_desc,edad,departamento,provincia,distrito,flag_tercio_per_ant,flag_quinto_per_ant,flag_decimo_per_ant,unidad_negocio,modalidad,grupo_tipo_ingreso,es_verano,flag_mtr_per_ant,max_mtr_continuas_reg,pct_beca,flag_beca,desc_beca,n_per_mtr_hist,n_per_mtr_hist_reg,n_per_mtr_hist_verano,flag_mtr_verano,estado_matricula_new,cant_creds_aprob_acum,cant_creds_carrera,cant_cursos_matr,documento,ponderado_acum,ponderado_ant,avance_creds_carrera
0,202401,N00001409,2024-08-15,NO_DETERMINADO,DEH-WA,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,TML,NO_DETERMINADO,48,LA LIBERTAD,TRUJILLO,HUANCHACO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,1,6,0.0,0,None,8,8,0,0,REG_05,214.0,241.0,5.0,18165456,14.15,15.90,0.887967
1,202401,N00001515,2024-08-15,NO_DETERMINADO,DEH-WV,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,VIR,NO_DETERMINADO,47,LA LIBERTAD,TRUJILLO,LA LIBERTAD/TRUJILLO/VICTOR LARCO HERRERA,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.0,0,None,1,1,0,0,NEW_00,NaN,240.0,5.0,18207703,NaN,NaN,0.000000
2,202401,N00002001,2024-08-15,NO_DETERMINADO,DEH-WA,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,TML,NO_DETERMINADO,51,LA LIBERTAD,TRUJILLO,LA LIBERTAD/TRUJILLO/TRUJILLO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,1,3,0.5,1,BECA EXCEL ACADE.50%,4,3,1,1,REG_02,41.0,240.0,5.0,03369952,18.51,19.00,0.170833
3,202401,N00002296,2024-08-15,NO_DETERMINADO,DEH-WA,DERECHO,DER,NaN,DERECHO Y CIENCIAS POLITICAS,NO_DETERMINADO,TML,NO_DETERMINADO,44,LA LIBERTAD,TRUJILLO,TRUJILLO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,1,5,0.0,0,None,5,5,0,0,REG_04,150.0,240.0,3.0,40636072,15.75,16.78,0.625000
4,202401,N00002355,2024-08-15,NO_DETERMINADO,CIV-WA,ING. CIVIL,NO_DETERMINADO,NaN,INGENIERIA,ING,TSI,NO_DETERMINADO,44,LA LIBERTAD,TRUJILLO,TRUJILLO,0.0,0.0,0.0,WA,PRES,NO_DETERMINADO,0,1,5,0.0,0,None,6,5,1,1,REG_04,125.0,200.0,5.0,40722626,15.83,20.00,0.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510961,202502,N00521521,2026-01-15,NO_DETERMINADO,IND-WV,ING. INDUSTRIAL,ING_IND,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,22,LIMA,LIMA,LIMA/LIMA/LIMA,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.0,0,None,1,1,0,0,NEW_00,NaN,200.0,8.0,72443933,NaN,NaN,0.000000
510962,202502,N00521522,2026-01-15,NO_DETERMINADO,IND-WV,ING. INDUSTRIAL,ING_IND,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,32,LIMA,HUAURA,LIMA/HUAURA/HUACHO,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.0,0,None,1,1,0,0,NEW_00,NaN,200.0,8.0,48609870,NaN,NaN,0.000000
510963,202502,N00521523,2026-01-15,NO_DETERMINADO,CON-WV,CONTABILIDAD Y FINANZAS,CONT_FIN,NaN,NEGOCIOS,NEG,VIR,NO_DETERMINADO,20,LIMA,LIMA,LIMA/LIMA/INDEPENDENCIA,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.0,0,None,1,1,0,0,NEW_00,NaN,200.0,7.0,73062776,NaN,NaN,0.000000
510964,202502,N00521524,2026-01-15,NO_DETERMINADO,SIC-WV,ING. DE SIST. COMPUTACIONALES,NO_DETERMINADO,NaN,INGENIERIA,ING,VIR,NO_DETERMINADO,28,LIMA,LIMA,LIMA/LIMA/SAN MARTIN DE PORRES,0.0,0.0,0.0,WV,VIRT,NO_DETERMINADO,0,0,1,0.0,0,None,1,1,0,0,NEW_00,NaN,200.0,8.0,70836962,NaN,NaN,0.000000


# DATASET FINAL

In [ ]:
df_final = df.copy()

cols_final = [
    'periodo',
    'cod_alumno',
    'documento',
    'edad',
    'departamento',
    'provincia',
    'distrito',
    'unidad_negocio',
    'estado_matricula_new',
    'modalidad',
    'campus',
    'facultad',
    'cod_carrera',
    'carrera',
    'flag_beca',
    'cant_cursos_matr',
    'ponderado_acum',
    'ponderado_ant',
    'cant_creds_aprob_acum',
    'cant_creds_carrera',
    'avance_creds_carrera',
    'flag_tercio_per_ant',
    'flag_quinto_per_ant',
    'flag_decimo_per_ant',
    'flag_mtr_per_ant',
    'n_per_mtr_hist_reg',
    'n_per_mtr_hist_verano',
]
df_final = df_final[ cols_final ].copy()

df_final

,periodo,cod_alumno,documento,edad,departamento,provincia,distrito,unidad_negocio,estado_matricula_new,modalidad,campus,facultad,cod_carrera,carrera,flag_beca,cant_cursos_matr,ponderado_acum,ponderado_ant,cant_creds_aprob_acum,cant_creds_carrera,avance_creds_carrera,flag_tercio_per_ant,flag_quinto_per_ant,flag_decimo_per_ant,flag_mtr_per_ant,max_mtr_continuas_reg,n_per_mtr_hist,n_per_mtr_hist_reg,n_per_mtr_hist_verano,flag_mtr_verano
0,202401,N00001409,18165456,48,LA LIBERTAD,TRUJILLO,HUANCHACO,WA,REG_05,PRES,TML,DERECHO Y CIENCIAS POLITICAS,DEH-WA,DERECHO,0,5.0,14.15,15.90,214.0,241.0,0.887967,0.0,0.0,0.0,1,6,8,8,0,0
1,202401,N00001515,18207703,47,LA LIBERTAD,TRUJILLO,LA LIBERTAD/TRUJILLO/VICTOR LARCO HERRERA,WV,NEW_00,VIRT,VIR,DERECHO Y CIENCIAS POLITICAS,DEH-WV,DERECHO,0,5.0,NaN,NaN,NaN,240.0,0.000000,0.0,0.0,0.0,0,1,1,1,0,0
2,202401,N00002001,03369952,51,LA LIBERTAD,TRUJILLO,LA LIBERTAD/TRUJILLO/TRUJILLO,WA,REG_02,PRES,TML,DERECHO Y CIENCIAS POLITICAS,DEH-WA,DERECHO,1,5.0,18.51,19.00,41.0,240.0,0.170833,0.0,0.0,0.0,1,3,4,3,1,1
3,202401,N00002296,40636072,44,LA LIBERTAD,TRUJILLO,TRUJILLO,WA,REG_04,PRES,TML,DERECHO Y CIENCIAS POLITICAS,DEH-WA,DERECHO,0,3.0,15.75,16.78,150.0,240.0,0.625000,0.0,0.0,0.0,1,5,5,5,0,0
4,202401,N00002355,40722626,44,LA LIBERTAD,TRUJILLO,TRUJILLO,WA,REG_04,PRES,TSI,INGENIERIA,CIV-WA,ING. CIVIL,0,5.0,15.83,20.00,125.0,200.0,0.625000,0.0,0.0,0.0,1,5,6,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510961,202502,N00521521,72443933,22,LIMA,LIMA,LIMA/LIMA/LIMA,WV,NEW_00,VIRT,VIR,INGENIERIA,IND-WV,ING. INDUSTRIAL,0,8.0,NaN,NaN,NaN,200.0,0.000000,0.0,0.0,0.0,0,1,1,1,0,0
510962,202502,N00521522,48609870,32,LIMA,HUAURA,LIMA/HUAURA/HUACHO,WV,NEW_00,VIRT,VIR,INGENIERIA,IND-WV,ING. INDUSTRIAL,0,8.0,NaN,NaN,NaN,200.0,0.000000,0.0,0.0,0.0,0,1,1,1,0,0
510963,202502,N00521523,73062776,20,LIMA,LIMA,LIMA/LIMA/INDEPENDENCIA,WV,NEW_00,VIRT,VIR,NEGOCIOS,CON-WV,CONTABILIDAD Y FINANZAS,0,7.0,NaN,NaN,NaN,200.0,0.000000,0.0,0.0,0.0,0,1,1,1,0,0
510964,202502,N00521524,70836962,28,LIMA,LIMA,LIMA/LIMA/SAN MARTIN DE PORRES,WV,NEW_00,VIRT,VIR,INGENIERIA,SIC-WV,ING. DE SIST. COMPUTACIONALES,0,8.0,NaN,NaN,NaN,200.0,0.000000,0.0,0.0,0.0,0,1,1,1,0,0


In [ ]:
# agregar cantidad de periodos seguidos matriculados actual, no solo el maximo
# quedar

In [45]:
pq.write_table( pa.Table.from_pandas( df_final ) , ruta_output_pq )